In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_185572\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_01440s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-425.041608,-77.950447,0,2,2090.529471,3821.000000,0.499131,0.000061,1,-425.041608,-1436,-26.737013,-894.089744,-5.826496,-73.992142,0.508633
1,2,1,1,NaN,-687.554293,-83.046325,0,3,2086.313863,3820.966777,0.328141,0.574388,1,-687.554293,-1436,-24.611071,-894.075595,-5.361884,-73.992423,0.762949
2,3,1,1,NaN,-395.425159,-71.558915,0,2,1926.524284,3819.000000,0.499410,0.000000,0,-395.425159,None,55.971649,-893.182484,12.109001,-73.776935,0.508633
3,4,1,1,NaN,-427.401395,-83.874285,0,2,1994.550491,3813.550491,0.497444,0.497444,0,-427.401395,None,21.669561,-890.392278,4.546983,-73.370069,0.508633
4,5,1,1,NaN,-944.981250,-93.609713,2,4,1929.280999,3813.995890,0.259511,0.386099,0,-944.981250,None,54.584536,-890.657219,11.509380,-73.355528,1.017266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255549,799326,97,8,NaN,199.015209,31.849835,0,3,69.278621,356.373283,0.317010,0.342011,0,NaN,None,-756.014623,407.795041,-59.356241,23.075870,48.949968
1255550,799327,97,8,NaN,90.685372,30.952748,0,2,122.514348,370.514348,0.499794,0.499794,0,NaN,None,-540.974861,464.916291,-38.888898,25.685380,32.633312
1255551,799328,97,8,NaN,70.068150,28.968496,0,2,289.494932,387.000000,0.499974,0.000000,0,NaN,None,133.524514,531.508235,9.215292,29.380781,32.633312
1255552,799329,97,8,NaN,116.624086,27.849925,0,2,401.000000,409.467589,0.000000,0.498948,1,NaN,775696,583.936756,622.263546,50.819432,37.941382,32.633312


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 601597/601597 [2:05:09<00:00, 80.12it/s]  


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_01440s.pnds')

In [8]:
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,...,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s,Life
0,1,1,1,NaN,-425.041608,-77.950447,0,2,2090.529471,3821.000000,...,0.000061,1,-425.041608,-1436,-26.737013,-894.089744,-5.826496,-73.992142,0.508633,1
1,2,1,1,NaN,-687.554293,-83.046325,0,3,2086.313863,3820.966777,...,0.574388,1,-687.554293,-1436,-24.611071,-894.075595,-5.361884,-73.992423,0.762949,2
2,3,1,1,NaN,-395.425159,-71.558915,0,2,1926.524284,3819.000000,...,0.000000,0,-395.425159,None,55.971649,-893.182484,12.109001,-73.776935,0.508633,1
3,4,1,1,NaN,-427.401395,-83.874285,0,2,1994.550491,3813.550491,...,0.497444,0,-427.401395,None,21.669561,-890.392278,4.546983,-73.370069,0.508633,1
4,5,1,1,NaN,-944.981250,-93.609713,2,4,1929.280999,3813.995890,...,0.386099,0,-944.981250,None,54.584536,-890.657219,11.509380,-73.355528,1.017266,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255549,799326,97,8,NaN,199.015209,31.849835,0,3,69.278621,356.373283,...,0.342011,0,NaN,None,-756.014623,407.795041,-59.356241,23.075870,48.949968,1
1255550,799327,97,8,NaN,90.685372,30.952748,0,2,122.514348,370.514348,...,0.499794,0,NaN,None,-540.974861,464.916291,-38.888898,25.685380,32.633312,1
1255551,799328,97,8,NaN,70.068150,28.968496,0,2,289.494932,387.000000,...,0.000000,0,NaN,None,133.524514,531.508235,9.215292,29.380781,32.633312,1
1255552,799329,97,8,NaN,116.624086,27.849925,0,2,401.000000,409.467589,...,0.498948,1,NaN,775696,583.936756,622.263546,50.819432,37.941382,32.633312,1
